In [3]:
#Submission to Kaggle Titanic Competition
import pandas as pd
DF = pd.read_csv('train.csv')
test_DF = pd.read_csv('test.csv')
DF.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os

# Preprocessing
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, StandardScaler

# Splitting
from sklearn.model_selection import train_test_split

# Models
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis



# Metrics
from sklearn.metrics import accuracy_score, f1_score

# Grid Search
from sklearn.model_selection import GridSearchCV

In [5]:
DF.dtypes

PassengerId      object
HomePlanet       object
CryoSleep        object
Cabin            object
Destination      object
Age             float64
VIP              object
RoomService     float64
FoodCourt       float64
ShoppingMall    float64
Spa             float64
VRDeck          float64
Name             object
Transported        bool
dtype: object

In [6]:
DF.describe()

,Age,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck
count,8514.000000,8512.000000,8510.000000,8485.000000,8510.000000,8505.000000
mean,28.827930,224.687617,458.077203,173.729169,311.138778,304.854791
std,14.489021,666.717663,1611.489240,604.696458,1136.705535,1145.717189
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,19.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,27.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,38.000000,47.000000,76.000000,27.000000,59.000000,46.000000
max,79.000000,14327.000000,29813.000000,23492.000000,22408.000000,24133.000000


In [7]:
DF.isnull().sum()

PassengerId       0
HomePlanet      201
CryoSleep       217
Cabin           199
Destination     182
Age             179
VIP             203
RoomService     181
FoodCourt       183
ShoppingMall    208
Spa             183
VRDeck          188
Name            200
Transported       0
dtype: int64

In [8]:
PASSENGER_ID = test_DF[['PassengerId']]
DF.drop(['PassengerId', 'Cabin', 'Name'], axis=1, inplace=True)
test_DF.drop(['PassengerId', 'Cabin', 'Name'], axis=1, inplace=True)

In [9]:

labels = test_DF.columns
for col in labels:
    if col in ['Age', 'RoomService','FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']:
        DF[col].fillna(DF[col].median(), inplace=True)
        test_DF[col].fillna(DF[col].median(), inplace=True)
    else:
        DF[col].fillna(DF[col].mode()[0], inplace=True)
        test_DF[col].fillna(DF[col].mode()[0], inplace=True)
for col in labels:
    if DF[col].dtype == 'O':
        encoder = LabelEncoder()
        DF[col] = encoder.fit_transform(DF[col])
        test_DF[col] = encoder.transform(test_DF[col])
        
    elif DF[col].dtype == 'bool':
        DF[col] = DF[col].astype('int')
        test_DF[col] = test_DF[col].astype('int')
encoder = LabelEncoder()
DF['Transported'] = DF['Transported'].astype('int')
labels_MM = ['Age']
labels_SS = ['RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']

In [10]:
mm_scaler = MinMaxScaler()
ss_scaler = StandardScaler()

DF[labels_MM] = mm_scaler.fit_transform(DF[labels_MM])
test_DF[labels_MM] = mm_scaler.transform(test_DF[labels_MM])


DF[labels_SS] = ss_scaler.fit_transform(DF[labels_SS])
test_DF[labels_SS] = ss_scaler.transform(test_DF[labels_SS])



In [11]:
X, y =DF.drop('Transported', axis=1), np.array(DF[['Transported']]).reshape(-1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.10, stratify=y)

y

array([0, 1, 0, ..., 1, 0, 1])

In [12]:
model_accuracy = {}

In [13]:
ensemble_models = GaussianNB()
predictor_gnb = ensemble_models.fit(X_train, y_train)
y_pred_gnb = predictor_gnb.predict(X_test)
accuracy_gnb = accuracy_score(y_test, y_pred_gnb)
model_accuracy['naive_bayes'] = accuracy_gnb
print("Accuracy of Gaussian Naive Bayes :",accuracy_gnb)

Accuracy of Gaussian Naive Bayes : 0.6873563218390805


In [14]:
ensemble_models = LogisticRegression()
predictor_lr = ensemble_models.fit(X_train, y_train)
y_pred_lr = predictor_lr.predict(X_test)
accuracy_lr = accuracy_score(y_test, y_pred_lr)
model_accuracy['logistic_regression'] = accuracy_lr
print("Accuracy of Logistic Regression :",accuracy_lr)

Accuracy of Logistic Regression : 0.767816091954023


In [15]:
ensemble_models = SVC()
predictor_svc = ensemble_models.fit(X_train, y_train)
y_pred_svc = predictor_svc.predict(X_test)
accuracy_svc = accuracy_score(y_test, y_pred_svc)
model_accuracy['SVC'] = accuracy_svc
print("Accuracy of SVC :",accuracy_svc)

Accuracy of SVC : 0.7597701149425288


In [16]:
ensemble_models = DecisionTreeClassifier()
predictor_dtc = ensemble_models.fit(X_train, y_train)
y_pred_dtc = predictor_dtc.predict(X_test)
accuracy_dtc = accuracy_score(y_test, y_pred_dtc)
model_accuracy['decision_tree'] = accuracy_dtc
print("Accuracy of Decision Tree :",accuracy_dtc)


Accuracy of Decision Tree : 0.7528735632183908


In [17]:
ensemble_models = RandomForestClassifier()
predictor_rtc = ensemble_models.fit(X_train, y_train)
y_pred_rtc = predictor_rtc.predict(X_test)
accuracy_rtc = accuracy_score(y_test, y_pred_rtc)
model_accuracy['random_forest'] = accuracy_rtc
print("Accuracy of Random Tree :",accuracy_rtc)

Accuracy of Random Tree : 0.7827586206896552


In [18]:
ensemble_models = GradientBoostingClassifier()
predictor_gb = ensemble_models.fit(X_train, y_train)
y_pred_gb = predictor_gb.predict(X_test)
accuracy_gb = accuracy_score(y_test, y_pred_gb)
model_accuracy['gradient_boost'] = accuracy_gb
print("Accuracy of Gradient Boost :",accuracy_gb)

Accuracy of Gradient Boost : 0.7770114942528735


In [19]:
ensemble_models = MLPClassifier()
predictor_mlp = ensemble_models.fit(X_train, y_train)
y_pred_mlp = predictor_mlp.predict(X_test)
accuracy_mlp = accuracy_score(y_test, y_pred_mlp)
model_accuracy['MLP'] = accuracy_mlp
print("Accuracy of MLP :",accuracy_mlp)

Accuracy of MLP : 0.7735632183908046


In [20]:
ensemble_models = LinearDiscriminantAnalysis()
predictor_lda = ensemble_models.fit(X_train, y_train)
y_pred_lda = predictor_lda.predict(X_test)
accuracy_lda = accuracy_score(y_test, y_pred_lda)
model_accuracy['LDA'] = accuracy_lda
print("Accuracy of LDA :",accuracy_lda)

Accuracy of LDA : 0.7494252873563219


In [22]:
ensemble_models = KNeighborsClassifier()
predictor_knn = ensemble_models.fit(X_train, y_train)
y_pred_knn = predictor_knn.predict(X_test)
accuracy_knn = accuracy_score(y_test, y_pred_knn)
model_accuracy['KNN'] = accuracy_knn
print("Accuracy of KNN :",accuracy_knn)

Accuracy of KNN : 0.7471264367816092


c:\Users\loop9\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)


In [23]:
ensemble_models = AdaBoostClassifier()
predictor_ada = ensemble_models.fit(X_train, y_train)
y_pred_ada = predictor_ada.predict(X_test)
accuracy_ada = accuracy_score(y_test, y_pred_ada)
model_accuracy['ADA'] = accuracy_ada
print("Accuracy of ADA :",accuracy_ada)

Accuracy of ADA : 0.7770114942528735


In [24]:
model_accuracy

{'naive_bayes': 0.6873563218390805,
 'logistic_regression': 0.767816091954023,
 'SVC': 0.7597701149425288,
 'decision_tree': 0.7528735632183908,
 'random_forest': 0.7827586206896552,
 'gradient_boost': 0.7770114942528735,
 'MLP': 0.7735632183908046,
 'LDA': 0.7494252873563219,
 'KNN': 0.7471264367816092,
 'ADA': 0.7770114942528735}

In [28]:
voting_ensemble = VotingClassifier(estimators= [('GB', predictor_gb), ('MLP', predictor_mlp), ('SVC', predictor_svc), ('GNB',predictor_gnb), ('LR',predictor_lr), ('DT', predictor_dtc), ('RT', predictor_rtc), ('LDA', predictor_lda), ('KNN', predictor_knn), ('ADA', predictor_ada)],voting = 'hard')

ensemble_predictor = voting_ensemble.fit(X_train, y_train)

y_pred = ensemble_predictor.predict(X_test)
print(f'Accuracy scores for the first ensemble: {accuracy_score(y_test, y_pred)}')

c:\Users\loop9\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


Accuracy scores for the first ensemble: 0.7724137931034483


c:\Users\loop9\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)


In [33]:
ensemble_predictor

VotingClassifier(estimators=[('GB', GradientBoostingClassifier()),
                             ('MLP', MLPClassifier()), ('SVC', SVC()),
                             ('GNB', GaussianNB()),
                             ('LR', LogisticRegression()),
                             ('DT', DecisionTreeClassifier()),
                             ('RT', RandomForestClassifier()),
                             ('LDA', LinearDiscriminantAnalysis()),
                             ('KNN', KNeighborsClassifier()),
                             ('ADA', AdaBoostClassifier())])

In [26]:
submission = ensemble_predictor.predict(test_DF)

data = {'PassengerId': np.array(PASSENGER_ID).reshape(len(PASSENGER_ID)),
        'Transported': np.array(submission).astype('bool')}

submission_df = pd.DataFrame(data=data).reset_index()
submission_df.drop(columns=['index'], inplace=True, axis=1)
# submission_df.to_csv("submission.csv", index=False)


In [ ]:
#Class Submission

In [ ]:
#Olof Persson's code

In [ ]:
#Parametric model

In [ ]:
#Non-parametric model

In [ ]:
#Koushik Reddy's code

In [ ]:
#Parametric model

In [ ]:
#Non-parametric model

In [ ]:
#Hao Dian's Code

In [ ]:
#Logistic Regression

In [ ]:
#Decision tree